Here we implement our triangle detection algorithm from class and compare it to a built-in method that is in the networkX library.

In [62]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [63]:
# Our triangleless graph.
G = nx.Graph()
G.add_nodes_from([1,2,3,4])
G.add_edges_from([[1,2],[2,3],[3,4],[4,1]])

In [64]:
# Our triangleless graph.
H = nx.Graph()
H.add_nodes_from([1,2,3,4])
H.add_edges_from([[1,2],[2,3],[3,1],[4,1]])

In [65]:
# NetworkX has a built-in function for finding the triangles.
# Returns a dictionary from the vertices to the number of triangles with that vertex.  
print(nx.triangles(G))
print(nx.triangles(H))

{1: 0, 2: 0, 3: 0, 4: 0}
{1: 1, 2: 1, 3: 1, 4: 0}


In [66]:
# We can wrap that up to get a practically built-in 'hasTriangles' function.
def hasTrianglesNX(G):
    return any(v > 0 for v in nx.triangles(G).values())

assert not hasTrianglesNX(G)
assert hasTrianglesNX(H)

In [67]:
# Let's make our own using the algorithm from class that was sped up with fast matrix mult.
# We will just call on numpy for matrix multiplication, I bet they got it pretty darn fast.

def hasTriangles(G):
    n = G.number_of_nodes()
    A = nx.adjacency_matrix(G).toarray()
    B = A@A # There is (in theory) our time dominating line.
    for i in range(n):
        for j in range(i+1,n):
            if A[i][j] != 0 and B[i][j] != 0: return True
    return False

assert not hasTriangles(G)
assert hasTriangles(H)

/var/folders/by/wvnsnkkd01170lymx2nfjq900000gn/T/ipykernel_45424/2225154015.py:6: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G).toarray()


For fun, let's see how these to functions compare to one another time-wise. That `toarray` call in our function is messing with all sorts of sparsity techniques that would generally be speeding things up in the background, so I imagine our function will be very slow.  We will generate a bunch of random graphs by drawing from [Erdos-Renyi distributions)](https://en.wikipedia.org/wiki/Erd%C5%91s%E2%80%93R%C3%A9nyi_model) (with random $p$ parameters).  

In [86]:
import timeit

trials = 30
vertices = 200
def test(f, trials, vertices):
    for _ in range(trials):
        p = np.random.rand()
        f(nx.erdos_renyi_graph(vertices,p))
        
start = timeit.default_timer()
test(hasTrianglesNX, trials, vertices)
stop = timeit.default_timer()
print('Time with built-in: ', stop - start)

start = timeit.default_timer()
test(hasTriangles, trials, vertices)
stop = timeit.default_timer()
print('Time with our function: ', stop - start)

Time with built-in:  7.901388159999897


/var/folders/by/wvnsnkkd01170lymx2nfjq900000gn/T/ipykernel_45424/2225154015.py:6: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G).toarray()


Time with our function:  8.55508600300027
